kaggle 地址： https://www.kaggle.com/datasets/wenruliu/adult-income-dataset

In [1]:
! which python

/Users/guozhiqi/anaconda3/envs/ml/bin/python


In [2]:
! conda info --envs

# conda environments:
#
base                     /Users/guozhiqi/Downloads/enter
                      *  /Users/guozhiqi/anaconda3/envs/ml



In [3]:
import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn import preprocessing
import copy
import os
import gc
import joblib
import copy
import pandas as pd
import numpy as np
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [25]:

df = pd.read_csv("../input/adult.csv")

In [26]:
df

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [6]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [7]:
df.dtypes

age                 int64
workclass          object
fnlwgt              int64
education          object
educational-num     int64
marital-status     object
occupation         object
relationship       object
race               object
gender             object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
income             object
dtype: object

In [8]:
df.income.value_counts()

<=50K    37155
>50K     11687
Name: income, dtype: int64

In [27]:
df1 = df.loc[:1000,::]

In [28]:
df1

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,59,Local-gov,662460,10th,6,Widowed,Prof-specialty,Unmarried,White,Female,0,0,15,United-States,<=50K
997,22,Private,152328,Some-college,10,Never-married,Farming-fishing,Not-in-family,White,Male,0,0,40,United-States,<=50K
998,40,Private,120277,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,45,United-States,<=50K
999,36,Private,180278,Bachelors,13,Never-married,Prof-specialty,Not-in-family,Black,Male,0,0,40,United-States,<=50K


In [29]:
# list of numerical columns
num_cols = [
    "fnlwgt",
    "age", "capital-gain", "capital-loss", "hours-per-week"
]

# map targets to 0s and 1s
target_mapping = {"<=50K": 0,
                   ">50K": 1}
df1.loc[:, "income"] = df1.income.map(target_mapping)

# all columns are features except kfold & income columns
features = [
    f for f in df1.columns if f not in ("income")
]

# fill all NaN values with NONE
# note that I am converting all columns to "strings"
# it doesnt matter because all are categories
for col in features:
    # do not encode the numerical columns
    if col not in num_cols:
        df1.loc[:, col] = df1[col].astype(str).fillna("NONE")



/Users/guozhiqi/anaconda3/envs/ml/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [30]:
df1

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,59,Local-gov,662460,10th,6,Widowed,Prof-specialty,Unmarried,White,Female,0,0,15,United-States,0
997,22,Private,152328,Some-college,10,Never-married,Farming-fishing,Not-in-family,White,Male,0,0,40,United-States,0
998,40,Private,120277,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,45,United-States,0
999,36,Private,180278,Bachelors,13,Never-married,Prof-specialty,Not-in-family,Black,Male,0,0,40,United-States,0


### label encoding 例子

In [32]:
# now its time to label encode the features
for col in features:
    if col not in num_cols:
        # initialize LabelEncoder for each feature column
        lbl = preprocessing.LabelEncoder()
        # fit label encoder on all data
        lbl.fit(df1[col])
        # transform all the data
        df1.loc[:, col] = lbl.transform(df1[col])

/Users/guozhiqi/anaconda3/envs/ml/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [33]:
df1

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,3,226802,1,13,4,7,3,2,1,0,0,40,26,0
1,38,3,89814,11,15,2,5,0,4,1,0,0,50,26,0
2,28,2,336951,7,3,2,11,0,4,1,0,0,40,26,1
3,44,3,160323,15,1,2,7,0,2,1,7688,0,40,26,1
4,18,0,103497,15,1,4,0,3,4,0,0,0,30,26,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,59,2,662460,0,12,6,10,4,4,0,0,0,15,26,0
997,22,3,152328,15,1,4,5,1,4,1,0,0,40,26,0
998,40,3,120277,9,4,2,12,0,4,1,0,0,45,26,0
999,36,3,180278,9,4,4,10,1,2,1,0,0,40,26,0


### Target Encoding


In [97]:
df_enc = pd.read_csv("../input/adult_folds.csv")

In [76]:
df_enc2 = df_enc.loc[:1000,::]

In [77]:
# map targets to 0s and 1s
target_mapping = {"<=50K": 0,
                   ">50K": 1}

df_enc.loc[:, "income"] = df_enc.income.map(target_mapping)

In [78]:
# all columns are features except income and kfold columns
features = [
    f for f in df_enc.columns if f not in ("kfold", "income") and f not in num_cols
]

In [79]:
# now its time to label encode the features
for col in features:
    if col not in num_cols:
        # initialize LabelEncoder for each feature column
        lbl = preprocessing.LabelEncoder()
        # fit label encoder on all data
        lbl.fit(df_enc[col])
        # transform all the data
        df_enc.loc[:, col] = lbl.transform(df_enc[col])

In [80]:
# fetch training and validation data
df_train = df_enc[df_enc.kfold != 1].reset_index(drop=True)
df_valid = df_enc[df_enc.kfold == 1].reset_index(drop=True)

In [81]:
df_train

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,kfold
0,59,4,184493,15,9,2,12,0,4,1,0,0,40,39,1,0
1,62,4,81116,15,9,0,1,1,4,1,0,1974,40,39,0,0
2,74,4,101590,14,14,6,1,1,2,0,0,0,20,39,0,0
3,36,0,187167,11,8,5,0,1,4,0,0,0,30,39,0,0
4,51,1,27166,11,8,2,13,0,4,1,0,0,40,39,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39068,27,4,250314,6,4,2,11,0,4,1,0,0,40,13,0,4
39069,39,6,327120,11,8,2,3,0,4,1,0,0,40,32,0,4
39070,67,4,219687,15,9,6,12,1,4,1,0,0,18,39,0,4
39071,50,4,330543,13,0,2,8,0,4,1,0,0,40,26,0,4


In [82]:
df_train.groupby('workclass')["income"].mean()

workclass
0    0.098804
1    0.387684
2    0.296549
3    0.000000
4    0.217711
5    0.557207
6    0.276864
7    0.268218
8    0.058824
Name: income, dtype: float64

In [83]:
 # for all feature columns, i.e. categorical columns
for column in features:
    # create dict of category:mean target
    mapping_dict = dict(df_train.groupby(column)["income"].mean())

    # column_enc is the new column we have with mean encoding
    df_valid.loc[
    :, column + "_enc"
    ] = df_valid[column].map(mapping_dict)


In [101]:
mapping_dict

{0: 0.27312775330396477,
 1: 0.22727272727272727,
 2: 0.36231884057971014,
 3: 0.28888888888888886,
 4: 0.05333333333333334,
 5: 0.21428571428571427,
 6: 0.047058823529411764,
 7: 0.13793103448275862,
 8: 0.058333333333333334,
 9: 0.37254901960784315,
 10: 0.375,
 11: 0.2865853658536585,
 12: 0.38461538461538464,
 13: 0.043478260869565216,
 14: 0.13559322033898305,
 16: 0.0,
 17: 0.2916666666666667,
 18: 0.375,
 19: 0.4,
 20: 0.36585365853658536,
 21: 0.3,
 22: 0.31868131868131866,
 23: 0.16049382716049382,
 24: 0.3170731707317073,
 25: 0.05263157894736842,
 26: 0.04862023653088042,
 27: 0.07692307692307693,
 28: 0.058823529411764705,
 29: 0.07894736842105263,
 30: 0.29707112970711297,
 31: 0.208955223880597,
 32: 0.20454545454545456,
 33: 0.11333333333333333,
 34: 0.1111111111111111,
 35: 0.20430107526881722,
 36: 0.4107142857142857,
 37: 0.18518518518518517,
 38: 0.08695652173913043,
 39: 0.24357073615783773,
 40: 0.09722222222222222,
 41: 0.3157894736842105}

In [84]:
df_train

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,kfold
0,59,4,184493,15,9,2,12,0,4,1,0,0,40,39,1,0
1,62,4,81116,15,9,0,1,1,4,1,0,1974,40,39,0,0
2,74,4,101590,14,14,6,1,1,2,0,0,0,20,39,0,0
3,36,0,187167,11,8,5,0,1,4,0,0,0,30,39,0,0
4,51,1,27166,11,8,2,13,0,4,1,0,0,40,39,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39068,27,4,250314,6,4,2,11,0,4,1,0,0,40,13,0,4
39069,39,6,327120,11,8,2,3,0,4,1,0,0,40,32,0,4
39070,67,4,219687,15,9,6,12,1,4,1,0,0,18,39,0,4
39071,50,4,330543,13,0,2,8,0,4,1,0,0,40,26,0,4


In [85]:
df_valid

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,...,kfold,workclass_enc,education_enc,educational-num_enc,marital-status_enc,occupation_enc,relationship_enc,race_enc,gender_enc,native-country_enc
0,34,2,234096,7,11,2,10,0,4,1,...,1,0.296549,0.256369,0.256369,0.443993,0.452107,0.446883,0.253652,0.303830,0.243571
1,54,2,137691,11,8,2,3,0,4,1,...,1,0.296549,0.158923,0.158923,0.443993,0.223765,0.446883,0.253652,0.303830,0.243571
2,52,1,221532,9,12,2,11,0,4,1,...,1,0.387684,0.410569,0.410569,0.443993,0.305304,0.446883,0.253652,0.303830,0.243571
3,46,4,213140,9,12,2,10,0,4,1,...,1,0.217711,0.410569,0.410569,0.443993,0.452107,0.446883,0.253652,0.303830,0.273128
4,41,4,344624,9,12,0,4,1,4,1,...,1,0.217711,0.410569,0.410569,0.100699,0.476308,0.104345,0.253652,0.303830,0.243571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9764,37,1,75073,7,11,4,11,1,2,1,...,1,0.387684,0.256369,0.256369,0.046205,0.305304,0.104345,0.123467,0.303830,0.243571
9765,35,4,328466,6,4,2,8,0,4,1,...,1,0.217711,0.057003,0.057003,0.443993,0.042418,0.446883,0.253652,0.303830,0.048620
9766,34,4,399386,8,10,2,3,0,4,1,...,1,0.217711,0.246793,0.246793,0.443993,0.223765,0.446883,0.253652,0.303830,0.243571
9767,54,6,155496,15,9,4,8,4,4,0,...,1,0.276864,0.190832,0.190832,0.046205,0.042418,0.057359,0.253652,0.109234,0.243571


In [98]:
def mean_target_encoding(data):
    # make a copy of dataframe
    df = copy.deepcopy(data)

    # list of numerical columns
    # list of numerical columns
    num_cols = [
        "fnlwgt",
        "age",
        "capital-gain",
        "capital-loss",
        "hours-per-week"
    ]

    # map targets to 0s and 1s
    target_mapping = {"<=50K": 0,
                       ">50K": 1}

    df.loc[:, "income"] = df.income.map(target_mapping)

    # all columns are features except income and kfold columns
    features = [
        f for f in df.columns if f not in ("kfold", "income") and f not in num_cols
    ]

    # fill all NaN values with NONE
    # note that I am converting all columns to "strings"
    # it doesnt matter because all are categories
    for col in features:
        # do not encode the numerical columns
        if col not in num_cols:
            df.loc[:, col] = df[col].astype(str).fillna("NONE")

    # now its time to label encode the features
    for col in features:
        if col not in num_cols:
            # initialize LabelEncoder for each feature column
            lbl = preprocessing.LabelEncoder()
            # fit label encoder on all data
            lbl.fit(df[col])
            # transform all the data
            df.loc[:, col] = lbl.transform(df[col])

    # a list to store 5 validation dataframes
    encoded_dfs = []

    # go over all folds
    for fold in range(5):
        # fetch training and validation data
        df_train = df[df.kfold != fold].reset_index(drop=True)
        df_valid = df[df.kfold == fold].reset_index(drop=True)

        # for all feature columns, i.e. categorical columns
        for column in features:
        # create dict of category:mean target
            mapping_dict = dict(df_train.groupby(column)["income"].mean())

            # column_enc is the new column we have with mean encoding
            df_valid.loc[
            :, column + "_enc"
            ] = df_valid[column].map(mapping_dict)

        # append to our list of encoded validation dataframes
        encoded_dfs.append(df_valid)

    # create full data frame again and return
    encoded_df = pd.concat(encoded_dfs, axis=0)

    return encoded_df


In [99]:
aa = mean_target_encoding(df_enc)

In [100]:
aa

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,...,kfold,workclass_enc,education_enc,educational-num_enc,marital-status_enc,occupation_enc,relationship_enc,race_enc,gender_enc,native-country_enc
0,59,4,184493,15,1,2,12,0,4,1,...,0,0.217595,0.186902,0.186902,0.445844,0.267222,0.448278,0.254358,0.303076,0.243741
1,62,4,81116,15,1,0,1,1,4,1,...,0,0.217595,0.186902,0.186902,0.101376,0.138920,0.101863,0.254358,0.303076,0.243741
2,74,4,101590,14,6,6,1,1,2,0,...,0,0.217595,0.739645,0.739645,0.089672,0.138920,0.101863,0.117412,0.111256,0.243741
3,36,0,187167,11,15,5,0,1,4,0,...,0,0.087703,0.157196,0.157196,0.063652,0.087427,0.101863,0.254358,0.111256,0.243741
4,51,1,27166,11,15,2,13,0,4,1,...,0,0.403296,0.157196,0.157196,0.445844,0.295533,0.448278,0.254358,0.303076,0.243741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9763,27,4,250314,6,11,2,11,0,4,1,...,4,0.217623,0.060956,0.060956,0.447326,0.316186,0.449933,0.253536,0.304113,0.029851
9764,39,6,327120,11,15,2,3,0,4,1,...,4,0.283037,0.159820,0.159820,0.447326,0.227142,0.449933,0.253536,0.304113,0.150000
9765,67,4,219687,15,1,6,12,1,4,1,...,4,0.217623,0.193149,0.193149,0.080740,0.267743,0.099722,0.253536,0.304113,0.243747
9766,50,4,330543,13,0,2,8,0,4,1,...,4,0.217623,0.000000,0.000000,0.447326,0.041847,0.449933,0.253536,0.304113,0.053735
